In [1]:
#Importing Packages I'll need
import pandas as pd
import numpy as np
import os
import scipy as sp
import re

# Initial Loading

In [2]:
#Loading the main Opportunity Atlas file. This will only work if the file below is in the same folder as this notebook
OA = pd.read_csv("big_data.nosync/tract_outcomes_early.csv")
OA.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5703) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,state,county,tract,kir_natam_female_p1,kir_natam_female_p25,kir_natam_female_p50,kir_natam_female_p75,kir_natam_female_p100,kir_natam_female_n,kir_natam_female_mean,...,jail_white_pooled_mean_se,kfr_white_pooled_mean_se,staytract_white_pooled_mean_se,staycz_white_pooled_mean_se,stayhome_white_pooled_mean_se,working_white_pooled_mean_se,kir_stycz_white_pooled_mean_se,kfr_stycz_white_pooled_mean_se,spouse_rk_white_pooled_mean_se,par_rank_white_pooled_mean_se
0,1,1,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007490,0.014829,0.020353,0.025752,0.019710,0.025348,0.018290,0.017890,0.018882,0.013778
1,1,1,20200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003828,0.020797,0.028821,0.041338,0.028252,0.029613,0.030380,0.028010,0.025328,0.021874
2,1,1,20300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002141,0.011726,0.016670,0.022392,0.015206,0.019161,0.015177,0.015285,0.016145,0.009353
3,1,1,20400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001777,0.010787,0.016027,0.019096,0.012477,0.016844,0.014253,0.014508,0.014396,0.009628
4,1,1,20500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000983,0.008789,0.012861,0.017491,0.010930,0.014396,0.011836,0.012748,0.010329,0.006667


#### Loading the Business Master File Data.

In [3]:
#First, 1989
BMF_1989 = pd.read_csv("big_data.nosync/bmf.bmf89.csv")

#Only California data
BMF_1989 = BMF_1989[BMF_1989['STATE'] == "CA"]

#Ading a year column
BMF_1989["Year"] = [1989]*len(BMF_1989)
BMF_1989.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,EIN,NAME,CITY,STATE,ZIP,SUBSECCD,ZIP5,MSA_NECH,PMSA,fipsold,...,stateOld1,zip5Old1,cityOld1,NteeOldB,Ntee2007a,Confidence2007a,RandNum,nteeFinal,nteeFinal1,Year
367,10319131,SIGMA DELTA PI,RIVERSIDE,CA,92517-0125,3.0,92517,4472.0,6780.0,NaN,...,CA,92517,RIVERSIDE,Z99,B83,A,0.39017,B83,B,1989
2659,20366251,HOLY SPIRIT RENEWAL MINISTRIES IN AMERICAN BAP...,PASADENA,CA,91104-2647,3.0,91104,4472.0,4480.0,NaN,...,CA,91104,PASADENA,Z99,X21,B,0.38955,X21,X,1989
4500,33045146,THE HEARING IMPAIRED YOUTH FOUNDATION INC,ANAHEIM,CA,92805-6218,3.0,92805,4472.0,5945.0,NaN,...,CA,92805,ANAHEIM,Z99,O12,A,0.08424,O12,O,1989
5813,42241214,CITIZENS RESEARCH FOUNDATION,LOS ANGELES,CA,90007-4377,3.0,90007,4472.0,4480.0,NaN,...,CA,90007,LOS ANGELES,Z99,W0520,B,0.95373,W0520,W,1989
6079,42386135,INTERNATIONAL SOCIETY FOR THE SYSTEMS SCIENCES,LOS ANGELES,CA,90089-0000,3.0,90089,4472.0,4480.0,NaN,...,CA,90089,LOS ANGELES,Z99,V30,A,0.22222,H99,H,1989


In [4]:
#Next 2015. I've downloaded all the 2015 files and am combining them to make sure all nonprofits that began and ended
#in 2015 are included.

#Once you've run this cell, don't run it again

#This makes an empty table
BMF_2015 = pd.DataFrame([])

#This goes through the individual files and adds them to the table
for filename in os.listdir('big_data.nosync/2015_bmf'):
     BMF_2015 = BMF_2015.append(pd.read_csv('big_data.nosync/2015_bmf/'+filename, low_memory=False))

#Only California Data
BMF_2015 = BMF_2015[BMF_2015['STATE'] == "CA"]
BMF_2015.head()

,ACCPER,ADDRESS,ASSETS,CITY,EIN,EPOST,FILER,FIPS,FISYR_IMAGE,FNDNCD,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
18,3.0,4102 MARLBOROUGH,0.0,SAN DIEGO,2296179.0,Z,N,6073.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X20,X,RE,RE,OT
19,12.0,217 N GRANT ST,0.0,SAN MATEO,3140260.0,Z,N,6081.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
89,12.0,4050 KATELLA AVE STE 101,0.0,LOS ALAMITOS,10104976.0,Z,N,6059.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X20,X,RE,RE,OT
119,12.0,514 SHATTO PL FL 3,0.0,LOS ANGELES,10137902.0,Z,Y,6037.0,2011.0,0.0,...,submaster990n,201412.0,NaN,Y,B,J40,J,HU,HU,HU
2495,12.0,3650 WHITE ALDER,0.0,SONOMA,10482703.0,Z,N,6097.0,2005.0,0.0,...,NaN,NaN,NaN,N,B,Y20,Y,MU,MU,OT


In [5]:
#Since the same org will appear several times, this will get rid of duplicates
#BMF_2015.drop_duplicates(inplace = True)

BMF_2015_unique = BMF_2015.groupby("EIN").first()

#Adding a year column with the year
BMF_2015_unique['Year'] = [2015]*len(BMF_2015_unique)

BMF_2015_unique.head()

,ACCPER,ADDRESS,ASSETS,CITY,EPOST,FILER,FIPS,FISYR_IMAGE,FNDNCD,FRCD,...,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5,Year
EIN,,,,,,,,,,,,,,,,,,,,,
2296179.0,3.0,4102 MARLBOROUGH,0.0,SAN DIEGO,Z,N,6073.0,NaN,10.0,6.0,...,NaN,NaN,N,A,X20,X,RE,RE,OT,2015
3140260.0,12.0,217 N GRANT ST,0.0,SAN MATEO,Z,N,6081.0,NaN,10.0,6.0,...,NaN,NaN,N,A,X21,X,RE,RE,OT,2015
10104976.0,12.0,4050 KATELLA AVE STE 101,0.0,LOS ALAMITOS,Z,N,6059.0,NaN,10.0,6.0,...,NaN,NaN,N,A,X20,X,RE,RE,OT,2015
10137902.0,12.0,514 SHATTO PL FL 3,0.0,LOS ANGELES,Z,Y,6037.0,2011.0,0.0,2.0,...,201412.0,0.0,Y,B,J40,J,HU,HU,HU,2015
10482703.0,12.0,3650 WHITE ALDER,0.0,SONOMA,Z,N,6097.0,2005.0,0.0,1.0,...,NaN,NaN,N,B,Y20,Y,MU,MU,OT,2015


#### The Business Master File is more granular than Opportunity Atlas, so we have to match BMF to the minimum shared granularity.

I originally wanted to do census tract, since this is the actual minimum shared granularity. However, since the 1989 BMF data does not include addresses, we cannot match the areas to census tracts easily. The next best thing is to work on the city level, and instead match opportutnity atlas data to cities.

In [6]:
city_tract = pd.io.stata.read_stata('city_name_to_tract.dta')
city_tract.head()

,state,county,tract,city,share_tract_in_city,maxval,max,state_abbr
0,1,1,20100,Prattville,1.0,1.0,1.0,AL
1,1,1,20200,Prattville,1.0,1.0,1.0,AL
2,1,1,20300,Prattville,1.0,1.0,1.0,AL
3,1,1,20400,Prattville,1.0,1.0,1.0,AL
4,1,1,20500,Prattville,1.0,1.0,1.0,AL


In [7]:
OA_city = OA.merge(city_tract, "left")

#Only California
OA_city = OA_city[OA_city['state_abbr'] == "CA"]

From here, I'm going to re-aggregate all the means according to city instead of census tract. I'll have to:
* Take each county mean value
* Multiply it by that mean's count or n (which gives the sum)
* Sum the expanded mean (now a sum) and the count by city
* Divide "summed sums" by "summed counts"

In [8]:
len(OA_city[pd.isnull(OA_city['city'])].iloc[:,0])

0

In [9]:
len(OA_city.iloc[:,0])

7989

Possibly there's a way to pool median/ percentile values but I'm not sure of it and we have plenty of averages so I'll use those.

There's also a way to pool SEs, but we'd be assuming that each tract's mean values are independent from each other. 

First I'll just pool the means :)

In [10]:
[x for x in OA_city.columns if "kid" in x]

['kid_hisp_male_n',
 'kid_hisp_female_n',
 'kid_white_male_n',
 'kid_white_female_n',
 'kid_black_male_n',
 'kid_black_female_n',
 'kid_natam_male_n',
 'kid_natam_female_n',
 'kid_asian_male_n',
 'kid_asian_female_n',
 'kid_pooled_pooled_n',
 'kid_pooled_male_n',
 'kid_pooled_female_n',
 'kid_white_pooled_n',
 'kid_black_pooled_n',
 'kid_asian_pooled_n',
 'kid_hisp_pooled_n',
 'kid_natam_pooled_n',
 'kid_pooled_pooled_blw_p50_n',
 'kid_white_pooled_blw_p50_n',
 'kid_black_pooled_blw_p50_n',
 'kid_hisp_pooled_blw_p50_n',
 'kid_asian_pooled_blw_p50_n',
 'kid_natam_pooled_blw_p50_n',
 'kid_pooled_male_blw_p50_n',
 'kid_white_male_blw_p50_n',
 'kid_black_male_blw_p50_n',
 'kid_hisp_male_blw_p50_n',
 'kid_asian_male_blw_p50_n',
 'kid_natam_male_blw_p50_n',
 'kid_pooled_female_blw_p50_n',
 'kid_white_female_blw_p50_n',
 'kid_black_female_blw_p50_n',
 'kid_hisp_female_blw_p50_n',
 'kid_asian_female_blw_p50_n',
 'kid_natam_female_blw_p50_n']

In [11]:
#All the columns that contain mean values
mean_col_names = [x for x in OA_city.columns if ("se" not in x) & ("_mean" in x) & ("par_rank_other" not in x)]
mean_col_names[0:5]

OA_tract_city = OA_city[['city', 'tract','state_abbr']]

#Sanity check that each corresponds to a column with a count in it
prefixes = np.array([])
for col in mean_col_names:
    pre = re.search(r"(.*)_mean$", col).group(1)
    prefixes = np.append(prefixes, pre)
    
    # The "par_rank" columns correspond to "kid" counts so some more maneuvering is needed
    if ("par_rank" in pre) & ("other" not in pre):
        suf = re.search(r"par_rank_(.*)", pre).group(1)
        OA_tract_city[pre + "_n"] = OA_city["kid_" + suf + "_n"]
        OA_tract_city[pre + "_sum"] = OA_city[col]*OA_city["kid_" + suf + "_n"]
    elif (pre + "_n") not in OA_city.columns:
        print(pre)
    else:
        OA_tract_city[pre + "_n"] = OA_city[pre + "_n"]
        OA_tract_city[pre + "_sum"] = OA_city[pre + "_n"]* OA_city[col]

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [12]:
OA_tract_city.head()

,city,tract,state_abbr,kir_natam_female_n,kir_natam_female_sum,jail_natam_female_n,jail_natam_female_sum,kfr_natam_female_n,kfr_natam_female_sum,kfr_top20_natam_female_n,...,marr_32_white_pooled_n,marr_32_white_pooled_sum,work_32_white_pooled_n,work_32_white_pooled_sum,work_26_white_pooled_n,work_26_white_pooled_sum,work_29_white_pooled_n,work_29_white_pooled_sum,work_24_white_pooled_n,work_24_white_pooled_sum
3546,Oakland,400100,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.0,71.092110,166.0,134.919752,146.0,118.950135,166.0,133.736731,102.0,75.811780
3547,Oakland,400200,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.0,40.049182,98.0,70.345816,83.0,67.010399,98.0,84.412300,52.0,40.359155
3548,Oakland,400300,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,172.0,67.620883,172.0,143.175081,155.0,131.654401,172.0,147.044174,97.0,81.867564
3549,Oakland,400400,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.0,34.054879,109.0,91.359912,92.0,77.345247,109.0,92.712600,58.0,46.272601
3550,Oakland,400500,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.0,12.721496,47.0,29.197191,38.0,23.977764,47.0,20.785639,27.0,20.514583


In [13]:
OA_grouped = OA_tract_city.groupby(['city', 'state_abbr']).sum()

#At this point the tract value doesn't make sense, so we'll remove it
OA_grouped.drop("tract", inplace = True, axis=1)
OA_grouped.head()

,,kir_natam_female_n,kir_natam_female_sum,jail_natam_female_n,jail_natam_female_sum,kfr_natam_female_n,kfr_natam_female_sum,kfr_top20_natam_female_n,kfr_top20_natam_female_sum,kir_top20_natam_female_n,kir_top20_natam_female_sum,...,marr_32_white_pooled_n,marr_32_white_pooled_sum,work_32_white_pooled_n,work_32_white_pooled_sum,work_26_white_pooled_n,work_26_white_pooled_sum,work_29_white_pooled_n,work_29_white_pooled_sum,work_24_white_pooled_n,work_24_white_pooled_sum
city,state_abbr,,,,,,,,,,,,,,,,,,,,,
Acton,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,890.0,487.745938,890.0,684.545869,781.0,650.714706,890.0,713.653686,523.0,457.713192
Adelanto,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,903.0,378.434259,903.0,659.337071,816.0,661.539488,903.0,650.014477,557.0,460.478325
Agoura Hills,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2845.0,1305.578893,2845.0,2217.275692,2419.0,2077.549662,2845.0,2310.552545,1624.0,1394.380914
Agua Dulce,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,436.0,218.288710,436.0,345.092734,379.0,313.037350,436.0,339.436571,262.0,226.158063
Aguanga,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,274.0,131.821612,274.0,178.996455,247.0,190.670183,274.0,200.939178,187.0,148.573928


In [14]:
for p in prefixes:
    n = OA_grouped[p + "_n"]
    s = OA_grouped[p + "_sum"]
    OA_grouped[p + "_mean"] = s/n
OA_grouped.head()

,,kir_natam_female_n,kir_natam_female_sum,jail_natam_female_n,jail_natam_female_sum,kfr_natam_female_n,kfr_natam_female_sum,kfr_top20_natam_female_n,kfr_top20_natam_female_sum,kir_top20_natam_female_n,kir_top20_natam_female_sum,...,kir_24_white_pooled_mean,marr_26_white_pooled_mean,marr_29_white_pooled_mean,kfr_29_white_pooled_mean,kir_29_white_pooled_mean,marr_32_white_pooled_mean,work_32_white_pooled_mean,work_26_white_pooled_mean,work_29_white_pooled_mean,work_24_white_pooled_mean
city,state_abbr,,,,,,,,,,,,,,,,,,,,,
Acton,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.560569,0.327264,0.464994,0.588517,0.560706,0.548029,0.769153,0.833181,0.801858,0.875169
Adelanto,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.493039,0.304453,0.385564,0.451404,0.446617,0.419086,0.730163,0.810710,0.719839,0.826712
Agoura Hills,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.560957,0.177945,0.335383,0.597835,0.604299,0.458903,0.779359,0.858846,0.812145,0.858609
Agua Dulce,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.609694,0.261612,0.409716,0.598228,0.596494,0.500662,0.791497,0.825956,0.778524,0.863199
Aguanga,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.475169,0.347813,0.383223,0.509786,0.489740,0.481101,0.653272,0.771944,0.733355,0.794513


In [15]:
BMF_2015_unique.columns = map(str.upper, BMF_2015_unique.columns)
BMF_2015_unique.columns

Index(['ACCPER', 'ADDRESS', 'ASSETS', 'CITY', 'EPOST', 'FILER', 'FIPS',
       'FISYR_IMAGE', 'FNDNCD', 'FRCD', 'GEN', 'INCOME', 'IRS990N', 'LEVEL1',
       'LEVEL2', 'LEVEL3', 'LEVEL4', 'MAJGRPB', 'MSA_NECH', 'NAICS', 'NAME',
       'NTEE1', 'NTEECC', 'NTEESRC', 'OUTNCCS', 'OUTREAS', 'PFFRCD', 'PMSA',
       'RULEDATE', 'RANDNUM', 'SEC_NAME', 'STATE', 'SUBSECCD', 'TAXPER',
       'UNNAMED: 0', 'ZFILER', 'ZIP5', 'CASSETS', 'CFILER', 'CFINSRC',
       'CTAXPER', 'CTOTREV', 'CZFILER', 'NTEECONF', 'NTEEFINAL', 'NTEEFINAL1',
       'NTMAJ10', 'NTMAJ12', 'NTMAJ5', 'YEAR'],
      dtype='object')

In [16]:
BMF_1989.columns = map(str.upper, BMF_1989.columns)
BMF_1989.columns

Index(['EIN', 'NAME', 'CITY', 'STATE', 'ZIP', 'SUBSECCD', 'ZIP5', 'MSA_NECH',
       'PMSA', 'FIPSOLD', 'FIPS', 'FNDNCD', 'ORGCD', 'ORGTYPCD', 'V_ERRORCD',
       'FILER', 'NTEECC', 'NTEECC4D', 'NTEECONF', 'NTEE1', 'LEVEL1', 'LEVEL2',
       'LEVEL3', 'LEVEL4', 'MAJGRPB', 'OUTNCCS', 'OUTREAS', 'NTEESRC',
       'NTMAJ10', 'NTMAJ12', 'NTMAJ5', 'ASSETS', 'INCOME', 'STATEOLD1',
       'ZIP5OLD1', 'CITYOLD1', 'NTEEOLDB', 'NTEE2007A', 'CONFIDENCE2007A',
       'RANDNUM', 'NTEEFINAL', 'NTEEFINAL1', 'YEAR'],
      dtype='object')

In [17]:
overlap = [x for x in BMF_2015_unique.columns if x in BMF_1989.columns]
overlap

['ASSETS',
 'CITY',
 'FILER',
 'FIPS',
 'FNDNCD',
 'INCOME',
 'LEVEL1',
 'LEVEL2',
 'LEVEL3',
 'LEVEL4',
 'MAJGRPB',
 'MSA_NECH',
 'NAME',
 'NTEE1',
 'NTEECC',
 'NTEESRC',
 'OUTNCCS',
 'OUTREAS',
 'PMSA',
 'RANDNUM',
 'STATE',
 'SUBSECCD',
 'ZIP5',
 'NTEECONF',
 'NTEEFINAL',
 'NTEEFINAL1',
 'NTMAJ10',
 'NTMAJ12',
 'NTMAJ5',
 'YEAR']

In [18]:
BMF_2015_clean = BMF_2015_unique[overlap]
BMF_1989_clean = BMF_1989[overlap]

## Write to computer

Lastly, we write the new files to the folder so we have cleaned versions to work on later

In [21]:
#OA_grouped.to_csv("big_data.nosync/clean/Opp_At_cleaned.csv")
#BMF_1989_clean.to_csv("big_data.nosync/clean/BMF_1989_cleaned.csv")
#BMF_2015_clean.to_csv("big_data.nosync/clean/BMF_2015_cleaned.csv")